# 합성곱 신경망 - CNN(Convolutional Neural Network)

이번 장의 주제는 합성곱 신경망이다.
CNN은 <span style='color:blue'>이미지 인식과 음성인식</span> 등 다양한 곳에서 사용되는데, 특히 <span style='background-color:yellow'>이미지 인식 분야</span>에서 딥러닝을 활용한 기법은 거의 CNN을 기초로 한다.

## 7.1 전체 구조

우선 CNN의 네트워크 구조를 보며 전체 틀을 이해해보자.
CNN도 지금까지 본 신경망과 같이 레고블록처럼 계층을 조합하여 만들 수 있다.<br/>
다만, <span style='color:red'>합성곱 계층</span>(convolutional layer)과 <span style='color:red'>풀링 계층</span>(pooling layer)이 새롭게 등장한다.

지금까지 본 신경망은 인접하는 계층의 모든 뉴런과 결합되어 있었다.<br/>
이를 <span style='color:blue'>완전연결</span>(fully-connected)이라고 하며, 완전히 연결된 계층을 <span style='color:blue'>Affine 계층</span>이라는 이름으로 구현했다.<br/>
이 Affine 계층을 사용하면, 가령 층이 5개인 완전연결 신경망은 다음 그림과 같이 구현할 수 있다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2F8X0l1%2FbtqKbVUkcYl%2FX4tJ2i0JAX2pRmLh5KLkFK%2Fimg.png" align="left" height=500 width=500>

그림과 같이 완전연결 신경망은 Affine 계층 뒤에 활성화 함수를 갖는 ReLU 계층(혹은 Sigmoid 계층)이 이어진다.<br/>
이 그림에서는 Affine-ReLU 조합이 4개가 쌓였고, 마지막 5번째 층은 Affine 계층에 이어 소프트맥스 계층에서 최종 결과(확률)를 출력한다.

그럼 CNN의 구조는 어떻게 다를까? 아래 그림이 CNN의 예시이다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FDKfqW%2FbtqJ1Mq3jh8%2FRM4ErDN0kSPsSdB94dvup0%2Fimg.png" align="left" height=500 width=500>

그림과 같이 CNN에서는 새로운 <span style='color:red'>합성곱 계층</span>(Conv)과 <span style='color:red'>풀링 계층</span>(Pooling)이 추가된다.<br/>
CNN의 계층은 <span style='color:blue'>'Conv - ReLU - (Pooling)'</span> 흐름으로 연결된다.(풀링 계층은 생략하기도 한다.)

CNN에서 주목할 또 다른 점은 출력에 가까운 층에서는 지금까지의 <span style='color:blue'>'Affine - ReLU'</span> 구성을 사용할 수 있다는 것이다.<br/>
또, 마지막 출력 계층에서는 <span style='color:blue'>'Affine - Softmax'</span>조합을 그대로 사용한다.<br/>
이상은 일반적인 CNN에서 흔히 볼 수 있는 구성이다.

## 7.2 합성곱 계층

CNN에서는 <span style='color:red'>패딩</span>(Padding), <span style='color:red'>스트라이드</span>(Stride) 등 CNN 고유의 용어가 등장한다.<br/>
또, 각 계층 사이에는 <span style='color:red'>3차원 데이터같이 입체적인 데이터가 흐른다는 점</span>에서 완전연결 신경망과 다르다.<br/>
우선 이번 절에서는 CNN에서 사용하는 합성곱 계층의 구조를 차분히 살펴보자.

### 7.2.1 완전연결 계층의 문제점

지금까지 본 완전연결 신경망에서는 완전연결 계층(Affine 계층)을 사용했다.<br/>
완전연결 계층에서는 인접하는 계층의 뉴런이 <span style='color:blue'>모두 연결</span>되고 출력의 수는 <span style='color:blue'>임의로</span> 정할 수 있다.

완전연결 계층의 문제점은 <span style='color:red'>'데이터의 형상이 무시된다'</span>는 사실이다.<br/>
입력 데이터가 이미지인 경우를 보면, 이미지는 통상적으로 (세로, 가로, 채널(색상))으로 구성된 3차원 데이터이다.<br/>
그러나 완전연결 계층에 입력할 때는 <span style='color:red'>3차원 데이터를 평평한 1차원 데이터로 평탄화</span>해줘야 한다.

이미지는 3차원 형상이며, 이 형상에는 소중한 공간적 정보가 담겨있다.<br>
예를 들어 공간적으로 가까운 픽셀은 값이 비슷하거나, RGB의 각 채널은 서로 밀접하게 관련되어있거나, 거리가 먼 픽셀끼리는 별 연관이 없는 등,<br/>
3차원 속에서 의미를 갖는 본질적인 패턴이 숨어 있을 것이다.

하지만 <span style='background-color:#ffdce0'>완전연결 계층은 형상을 무시하고 모든 입력 데이터를 동등한 뉴런으로 취급하여 형상에 담긴 정보를 살릴 수 없다.<br/>

한편 합성곱 계층은 형상을 유지한다.<br/>
이미지도 3차원 데이터로 입력받으며, 마찬가지로 다음 계층에도 3차원 데이터로 전달한다.<br/>
<span style='background-color:#ffdce0'>그래서 CNN에서는 이미지처럼 형상을 가진 데이터를 제대로 이해할 가능성이 있는 것이다.</span>

CNN에서는 합성곱 계층의 입출력 데이터를 <span style='background-color:#ffdce0'>특징 맵(feature map)</span>이라고도 한다.<br/>
합성곱 계층의 입력 데이터를 <span style='background-color:#ffdce0'>입력 특징 맵</span>, 출력 데이터를 <span style='background-color:#ffdce0'>출력 특징 맵</span>이라고 하는 식이다.<br/>
이 책에서는 입출력 데이터와 특징 맵을 같은 의미로 사용한다.

### 7.2.2 합성곱 연산

합성곱 계층에서의 <span style='background-color:#ffdce0'>합성곱 연산</span>을 처리한다.<br/>
합성곱 연산은 이미지 처리에서 말하는 <span style='background-color:#ffdce0'>필터 연산</span>에 해당한다.<br/>
다음 예시를 보자.

<img src="https://compmath.korea.ac.kr/appmath2021/_images/cnn_op.png" align="left" height=500 width=500>

그림과 같이 합성곱 연산은 입력 데이터에 필터를 적용한다.<br/>
이 예에서는 입력데이터는 (세로 * 가로) 방향의 형상을 가졌고, 필터 역시 (세로 * 가로) 방향의 차원을 갖는다.<br/>
데이터와 필터의 형상을 (높이, 너비)로 표기하며, 이 예에서는 입력은 (4, 4), 필터는 (3, 3), 출력은 (2, 2)가 된다.<br/>
문헌에 따라 필터를 <span style='background-color:#ffdce0'>커널</span>이라 칭하기도 한다.

<img src="https://compmath.korea.ac.kr/appmath2021/_images/cnn_op_order.png" align="left" height=500 width=500>

합성곱 연산은 필터의 <span style='background-color:#ffdce0'>윈도우</span>를 일정 간격으로 이동해가며 입력 데이터에 적용한다.<br/>
여기서 <span style='background-color:#ffdce0'>윈도우</span>는 그림의 회색 3X3 부분을 가리킨다.<br/>
이 그림에서 보듯 입력과 필터에서 대응하는 원소끼리 곱한 후 그 총합을 구한다.(이 계산을 <span style='background-color:#ffdce0'>단일 곱셈-누산</span>(Fused Multiply-Add, FMA)이라고 한다.)<br/>
그리고 그 결과를 출력의 해당 장소에 저장한다.<br/>
이 과정을 모든 장소에서 수행하면 합성곱 연산의 출력이 완성된다.

완전연결 신경망에는 가중치 매개변수와 편향이 존재하는데, <span style='background-color:#ffdce0'>CNN에서는 필터의 매개변수가 가중치에 해당한다.</span><br/>
그리고 <span style='background-color:#fff5b1'>편향도 존재한다.</span><br/>
아래 그림과 같이 <span style='background-color:#fff5b1'>편향은 필터를 적용한 후의 데이터에 더해진다.</span><br/>
그리고 <span style='background-color:#fff5b1'>편향은 항상 하나만 존재한다.</span><br/>
그 하나의 값을 필터를 적용한 모든 원소에 더하는 것이다.

<img src="https://compmath.korea.ac.kr/appmath2021/_images/cnn_op_order_bias.png" align="left" width=500 height=500>

### 7.2.3 패딩

패딩(Padding)은 <span style='background-color:#ffdce0'>합성곱 연산을 수행하기전에 입력 데이터 주변을 특정 값(0)으로 채우는 것이다.</span><br/>
패딩은 합성곱 연산에서 자주 이용하는 기법이다.

<img src="https://compmath.korea.ac.kr/appmath2021/_images/cnn_padding.png" align="left" width=500 height=500>

그림과 같이 처음에 크기가 (4, 4)인 입력 데이터에 패딩이 추가되어 (6, 6)이 된다.<br/>
이 입력에 (3, 3) 크기의 필터를 걸면 (4, 4)크기의 출력 데이터가 생성된다.<br/>
이 예에서는 패딩의 폭을 1로 설정했지만 원하는 정수로 정할 수도 있다.<br/>
<span style='background-color:#ffdce0'>패딩은 주로 출력 크기를 조정할 목적으로 사용한다.</span>

### 7.2.4 스트라이드

<span style='background-color:#ffdce0'>필터를 적용하는 위치의 간격을 스트라이드(Stride)라고 한다.</span><br/>
지금까지 본 예는 모두 스트라이드가 1이었지만, 예를들어 스트라이드를 2로 하면 필터를 적용하는 윈도우가 두 칸씩 이동한다.

<img src="https://compmath.korea.ac.kr/appmath2021/_images/cnn_stride.png" align="left" width=500 height=500>

크기가 (7, 7)인 입력 데이터에 스트라이드를 2로 설정한 필터를 적용하면 출력이 (3, 3)이 된다.<br/>
<span style='background-color:#ffdce0'>이처럼 스트라이드를 키우면 출력 크기는 작아진다.</span> 한편, 패딩을 크게 하면 출력 크기가 커졌다.<br/>
이러한 관계를 수식화하면 다음과 같다.

입력 크기를 $(H, W)$, 필터 크기를 $(FH, FW)$, 출력 크기를 ($OH, OW$), 패딩을 $P$, 스트라이드를 $S$라 하면 출력 크기는 다음 식으로 계산한다.

### $OH = {{H + 2P - FH}\over{S}} + 1$<br/>
### $OW = {{W + 2P - FW}\over{S}} + 1$

 단, 위 식의 ${{H + 2P - FH}\over{S}}$와 ${{W + 2P - FW}\over{S}}$가 정수로 니눠떨어지는 값이어야 한다는 점에 주의해야한다.

### 7.2.5 3차원 데이터의 합성곱 연산

지금까지는 2차원 형상을 다루는 합성곱 연산을 살펴봤다.<br/>
그러나 이미지만 해도 세로, 가로에 더해서 채널까지 고려한 3차원 데이터이다.<br/>
이번 절에서는 조금 전과 같은 순서로, 채널까지 고려한 3차원 데이터를 다루는 합성곱 연산을 살펴보자.

<img src="https://compmath.korea.ac.kr/appmath2021/_images/cnn_3d.png" align="left" width=500 height=500>

2차원일 때와 비교하면 길이 방향(채널 방향)으로 특징 맵이 늘어났다.<br/>
채널 쪽으로 특징 맵이 여러 개 있다면 입력 데이터와 필터의 합성곱 연산을 채널마다 수행하고, 그 결과를 더해서 하나의 출력을 얻는다.

<img src="https://compmath.korea.ac.kr/appmath2021/_images/cnn_3d_order.png" align="left" width=500 height=500>

3차원의 합성곱 연산에서 주의할 점은 <span style='background-color:#ffdce0'>입력 데이터의 채널 수와 필터의 채널 수가 같아야 한다는 것이다.</span><br/>
한편, 필터 자체의 크기는 원하는 값으로 설정할 수 있다.(단, 모든 채널의 필터가 같은 크기여야 한다.)<br/>

### 7.2.6 블록으로 생각하기

3차원의 합성곱 연산은 데이터와 필터를 직육면체 블록이라고 생각하면 쉽다.

<img src="https://compmath.korea.ac.kr/appmath2021/_images/cnn_block.png" align="left" width=500 height=500>


* C : 채널 수, H : 높이, W : 너비, FH : 필터 높이, FW : 필터 너비

위 예시에서  출력 데이터는 한 장의 특징 맵(다른 말로 하면 채널이 1개인 특징 맵)이다.<br/>
<span style='background-color:#ffdce0'>합성곱 연산의 출력으로 다수의 채널을 내보내려면 필터(가중치)를 다수 사용해야 한다.</span> 그림으로는 아래와 같다.

<img src="https://compmath.korea.ac.kr/appmath2021/_images/cnn_block_batch.png" align="left" width=500 height=500>

그림과 같이 <span style='background-color:#ffdce0'>필터를 FN개 적용하면 출력 맵도 FN개가 생성된다.</span><br/>
그리고 그 FN개의 맵을 모으면 형상이 (FN, OH, OW)인 블록이 완성된다.<br/>
이 완성된 블록을 다음 계층으로 넘기겠다는것이 CNN의 처리 흐름이다.

이상에서 보듯 합성곱 연산에서는 필터의 수도 고려해야 한다.<br/>
그런 이유로 <span style='background-color:#fff5b1'>필터의 가중치 데이터는 4차원 데이터이며 (출력 채널 수, 입력 채널 수, 높이, 너비) 순으로 쓴다.</span><br/>
예를 들어 채널 수가 3, 크기 5 * 5인 필터가 20개 있다면 (20, 3, 5, 5)로 쓴다.

합성곱 연산에서도 편향이 쓰인다.<br/>
위 그림에 편향을 더한 모습은 아래와 같다.

<img src="https://compmath.korea.ac.kr/appmath2021/_images/cnn_block_bias.png" align="left" width=600 height=500>

그림에서 보듯이 편향은 채널 하나에 값 하나씩으로 구성된다.<br/>
이 예에서는 편향의 형상은 (FN, 1, 1)이고, 필터의 출력 결과의 형상은 (FN, OH, OW)이다.<br/>
이 두 블록을 더하면 편향의 각 값이 필터의 출력인 (FN, OH, OW) 블록의 대응 채널의 원소 모두에 더해진다.

### 7.2.7 배치 처리

신경망 처리에서 입력데이터를 한 덩어리로 묶어 배치로 처리했다. 합성곱 연산도 마찬가지로 배치 처리를 지원하고자 한다.<br/>
그래서 각 계층을 흐르는 데이터의 차원을 하나 늘려 4차원 데이터로 저장한다.<br/>
데이터가 N개일 때 위 그림을 배치 처리한다면 데이터 형태는 아래와 같이 된다.

<img src="https://compmath.korea.ac.kr/appmath2021/_images/cnn_batch.png" align="left" width=600 height=500>

## 7.3 풀링 계층

<span style='background-color:#ffdce0'>풀링은 세로 $\cdot$ 가로 방향의 공간을 줄이는 연산이다.</span><br/>
아래와 같이 2$\times$2 영역을 원소 하나로 집약하여 공간 크기를 줄인다.

<img src="https://compmath.korea.ac.kr/appmath2021/_images/cnn_pooling.png" align="left" width=600 height=500>

위 그림은 2$\times$2 최대 풀링(Max Pooling)을 스트라이드 2로 처리하는 순서이다.<br/>
<span style='background-color:#ffdce0'>최대 풀링은 최댓값을 구하는 연산</span>으로, '2$\times$2'는 대상 영역의 <span style='background-color:#fff5b1'>크기</span>를 뜻한다.<br/>
즉, 2$\times$2 최대풀링은 그림과 같이 2$\times$2 크기의 영역에서 가장 큰 원소 하나를 꺼낸다.<br/>
또, 스트라이드는 이 예에서는 2로 설정했으므로 2$\times$2 윈도우가 원소 2칸 간격으로 이동한다.<br/>
<span style='background-color:#fff5b1'>참고로 풀링의 윈도우 크기와 스트라이드는 같은 값으로 설정하는 것이 보통이다.</span>

### 7.3.1 풀링 계층의 특징

#### 1. 학습해야 할 매개변수가 없다

풀링 계층은 합성곱 계층과 달리 <span style='background-color:#ffdce0'>학습해야 할 매개변수가 없다.</span><br/>
풀링은 대상 영역에서 최댓값이나 평균을 취하는 명확한 처리이므로 특별히 학습할 것이 없다.

#### 2. 채널 수가 변하지 않는다

풀링 연산은 입력 데이터의 채널 수 그대로 출력 데이터로 내보낸다.<br/>

<img src="https://compmath.korea.ac.kr/appmath2021/_images/cnn_pooling_channel.png" align="left" width=500 height=500>

#### 3. 입력의 변화에 영향을 적게 받는다(강건하다)

입력 데이터가 조금 변해도 풀링의 결과는 잘 변하지 않는다.<br/>
아래 그림은 입력 데이터의 차이를 풀링이 흡수해 사라지게 하는 모습을 보여준다.

<img src="https://compmath.korea.ac.kr/appmath2021/_images/cnn_pooling_robust.png" align="left" width=500 height=500>